In [2]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("taxi-analysis").getOrCreate()

22/04/05 18:27:56 WARN Utils: Your hostname, devkhk-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.30.1.27 instead (on interface en0)
22/04/05 18:27:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/05 18:27:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/05 18:27:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [14]:
trip_files = "/Users/devkhk/Documents/data-engineering-study/data/trips/*"
zone_file = "/Users/devkhk/Documents/data-engineering-study/data/taxi+_zone_lookup.csv"

In [18]:
trips_df = spark.read.csv(f"file:///{trip_files}", header=True, inferSchema=True)
zone_df = spark.read.csv(f"file:///{zone_file}", header=True, inferSchema=True)

In [23]:
trips_df.createOrReplaceTempView("trips")
zone_df.createOrReplaceTempView("zone")

In [25]:
trips_df.printSchema()
zone_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [53]:
# 두 테이블을 join해서 지역번호를 zone 이름으로 대체하는 테이블을 만든다.

query = """
SELECT 
    t.VendorID as vendor_id,
    TO_DATE(t.tpep_pickup_datetime) as pickup_date,
    TO_DATE(t.tpep_dropoff_datetime) as dropoff_date,
    HOUR(t.tpep_pickup_datetime) as pickup_time,
    HOUR(t.tpep_dropoff_datetime) as dropoff_time,
    t.passenger_count,
    t.tip_amount,
    t.tolls_amount,
    t.total_amount,
    pz.Zone as pickup_zone,
    dz.Zone as dropoff_zone
FROM
    trips t
    LEFT JOIN 
        zone pz
    ON
        t.PULocationID = pz.LocationID 
    LEFT JOIN
        zone dz
    ON
        t.DOLocationID = dz.LocationID
"""


In [54]:
comb_df = spark.sql(query)
comb_df.createOrReplaceTempView("comb")

In [55]:
comb_df.printSchema()

root
 |-- vendor_id: integer (nullable = true)
 |-- pickup_date: date (nullable = true)
 |-- dropoff_date: date (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- dropoff_time: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_zone: string (nullable = true)
 |-- dropoff_zone: string (nullable = true)



In [56]:
comb_df.select("pickup_date", "pickup_zone").show()

+-----------+--------------------+
|pickup_date|         pickup_zone|
+-----------+--------------------+
| 2021-03-01|                  NV|
| 2021-03-01|      Manhattanville|
| 2021-03-01|      Manhattanville|
| 2021-03-01|   LaGuardia Airport|
| 2021-03-01|        East Chelsea|
| 2021-03-01|Upper West Side S...|
| 2021-03-01|Penn Station/Madi...|
| 2021-03-01|         JFK Airport|
| 2021-03-01|   LaGuardia Airport|
| 2021-03-01|   LaGuardia Airport|
| 2021-03-01|         JFK Airport|
| 2021-03-01|     Lenox Hill East|
| 2021-03-01|Upper West Side S...|
| 2021-03-01|    Hamilton Heights|
| 2021-03-01|   East Harlem North|
| 2021-03-01|Upper West Side S...|
| 2021-03-01|         JFK Airport|
| 2021-03-01|Upper West Side S...|
| 2021-03-01|         JFK Airport|
| 2021-03-01|Sutton Place/Turt...|
+-----------+--------------------+
only showing top 20 rows



In [57]:
spark.sql("select pickup_date, pickup_time, pickup_zone, total_amount from comb").show()

+-----------+-----------+--------------------+------------+
|pickup_date|pickup_time|         pickup_zone|total_amount|
+-----------+-----------+--------------------+------------+
| 2021-03-01|          0|                  NV|         4.3|
| 2021-03-01|          0|      Manhattanville|         3.8|
| 2021-03-01|          0|      Manhattanville|         4.8|
| 2021-03-01|          0|   LaGuardia Airport|       70.07|
| 2021-03-01|          0|        East Chelsea|       11.16|
| 2021-03-01|          0|Upper West Side S...|       18.59|
| 2021-03-01|          0|Penn Station/Madi...|        43.8|
| 2021-03-01|          0|         JFK Airport|        32.3|
| 2021-03-01|          0|   LaGuardia Airport|       43.67|
| 2021-03-01|          0|   LaGuardia Airport|        46.1|
| 2021-03-01|          0|         JFK Airport|        45.3|
| 2021-03-01|          0|     Lenox Hill East|        19.3|
| 2021-03-01|          0|Upper West Side S...|        14.8|
| 2021-03-01|          0|    Hamilton He

In [58]:
spark.sql("select pickup_date, pickup_time, pickup_zone, total_amount from comb \
where pickup_time > 12").show()

+-----------+-----------+--------------------+------------+
|pickup_date|pickup_time|         pickup_zone|total_amount|
+-----------+-----------+--------------------+------------+
| 2021-02-28|         23|     Lenox Hill West|        9.36|
| 2021-02-28|         23|        Clinton East|        11.8|
| 2021-02-28|         23|Penn Station/Madi...|       17.16|
| 2021-02-28|         23|      Yorkville West|       17.88|
| 2021-02-28|         23|Penn Station/Madi...|        14.8|
| 2021-02-28|         23|        Clinton East|        14.3|
| 2021-02-28|         23|            Union Sq|        22.3|
| 2021-03-01|         22|         JFK Airport|        65.0|
| 2021-03-01|         13|   East Harlem South|         5.8|
| 2021-03-01|         13|         Murray Hill|         6.3|
| 2021-03-01|         13| UN/Turtle Bay South|        18.8|
| 2021-03-01|         13|    Manhattan Valley|         6.3|
| 2021-03-01|         13|            Union Sq|       12.36|
| 2021-03-01|         13|Upper East Side

In [60]:
spark.sql("select pickup_date, pickup_time, pickup_zone, total_amount from comb \
where pickup_date < '2020-12-31'").show() # 불필요한 데이터, 잘못된 데이터를 정리할 필요가 있다.

+-----------+-----------+--------------------+------------+
|pickup_date|pickup_time|         pickup_zone|total_amount|
+-----------+-----------+--------------------+------------+
| 2009-01-01|          0| Lincoln Square West|        21.3|
| 2008-12-31|         23|             Erasmus|         5.3|
| 2009-01-01|          0|        Clinton East|         9.3|
| 2009-01-01|          0|        Midtown East|        -3.3|
| 2009-01-01|          0|        Midtown East|         3.3|
| 2009-01-01|          0|Queensbridge/Rave...|         3.3|
| 2009-01-01|          0|         JFK Airport|        49.8|
| 2009-01-01|          1|Queensbridge/Rave...|         0.0|
| 2009-01-01|          0|Queensbridge/Rave...|         5.3|
| 2008-12-31|         23| Lincoln Square West|        13.0|
| 2008-12-31|         23|Sutton Place/Turt...|        9.96|
| 2008-12-31|         23|        Central Park|        12.8|
| 2008-12-31|         23|Upper East Side S...|         8.8|
| 2009-01-01|          0|      Yorkville